# Grid Search for Binary Classification

This notebook goes through applying the `grid_search` algorithm in FairLearn to a binary classification problem, where we also have a binary protected attribute.

The specific problem we consider a biased credit scoring problem. We assume that we have a collection of individuals characterised by two features - a credit score in the range $[0, 1]$ and a binary protected attribute from ${a_0, a_1}$. We also have a binary 'score' for each individual indicating whether or not they got a loan. This is determined by applying a threshold to their credit score, and to make the dataset unfair, we can set different thresholds for the two groups $a_0$ and $a_1$.

In this simple case, we make the protected attribute and the credit score available to both the learner and the model. This gives us a straightforward method of assessing fairness - look at the model, and see if zero weight is put on the protected attribute.

In [ ]:
import sys
sys.path.insert(0, "../")

from fairlearn.metrics import DemographicParity
from fairlearn.reductions import GridSearch
from fairlearn.reductions.grid_search.simple_quality_metrics import SimpleClassificationQualityMetric

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

We use the following function to create the input data. The credit scores for each population are uniformly distributed in the range $[0, 1]$, and we apply separate thresholds to each subpopulation to determine the score $Y \in {0, 1}$. We also add an extra constant feature to the data. This is needed to allow the toy learner we use to create a good fit.

In [ ]:
def simple_threshold_data(number_a0, number_a1,
                          a0_threshold, a1_threshold,
                          a0_label, a1_label):

    a0s = np.full(number_a0, a0_label)
    a1s = np.full(number_a1, a1_label)

    a0_scores = np.linspace(0, 1, number_a0)
    a1_scores = np.linspace(0, 1, number_a1)
    score_feature = np.concatenate((a0_scores, a1_scores), axis=None)

    A = np.concatenate((a0s, a1s), axis=None)

    Y_a0 = [x > a0_threshold for x in a0_scores]
    Y_a1 = [x > a1_threshold for x in a1_scores]

    Y = np.concatenate((Y_a0, Y_a1), axis=None)

    X = pd.DataFrame({"credit_score_feature": score_feature,
                      "protected_attribute_feature": A})
    return X, Y, A

We now use the above function to generate our dataset. We have 31 individuals with label $a_0$, and they only require a credit score of 0.2 to get the loan. In contrast, the 21 members of the $a_1$ population require a score of 0.7. The actual label values for $a_0$ and $a_1$ have to be numeric, but the actual values are not important. We set them to carefully chosen random numbers.

In [ ]:
num_samples_a0 = 31
num_samples_a1 = 21

a0_threshold= 0.2
a1_threshold = 0.7

a0_label = 2
a1_label = 3

X, Y, A = simple_threshold_data(num_samples_a0, num_samples_a1, a0_threshold, a1_threshold, a0_label, a1_label)

The following helper function plots the score $Y$ for each of the subpopulations in our data.

In [ ]:
import matplotlib.pyplot as plt
def plot_data(Xs, Ys):
    labels = np.unique(Xs["protected_attribute_feature"])
    
    for l in labels:
        label_string = str(l.item())
        mask = Xs["protected_attribute_feature"] == l
        plt.scatter(Xs[mask].credit_score_feature, Ys[mask], label=str("Label="+label_string))
        plt.xlabel("Credit Score")
        plt.ylabel("Got Loan")
        
    plt.legend()
    plt.show()

Plotting the input data, we can clearly see the bias against the $a_1$ population.

In [ ]:
plot_data(X, Y)

For this notebook, we use a very simple binary classifier. This performs linear regression on the input features (hence the requirement for the $a_0$ and $a_1$ labels to be numeric), and then checks whether or not the result is greater than 0.5. We can train this model on our biased data, and look at the weights the regression places on each feature. The fact that the `protected_attribute_feature` has non-zero weight tells us that we have a biased model (note that this is only true for this simple example - in the real world, fairness is more complicated).

In [ ]:
unfair_model = LogisticRegression(solver='liblinear', fit_intercept=True)
unfair_model.fit(X, Y, sample_weight=np.ones(len(Y)))

unfair_model.coef_

We can also plot out the predictions for this model. We can see that a few points have changed (which is not unexpected) but the bias definitely remains.

In [ ]:
Y_predict_unfair = unfair_model.predict(X)
plot_data(X, Y_predict_unfair)

## Reducing Unfairness with Grid Search

Now, we move on to attempting to reduce the unfairness in our model using the grid search. This tries a series of different models, parameterised by a Lagrange multiplier $\lambda_i$. For each value of $\lambda$, the algorithm reweights and relabels the input data, and trains a fresh model ($\lambda=0$ corresponds to the unaltered case). The grid search returns a list of dictionaries, each with two entries - `lagrange_multiplier` and `model`.

We start by telling the algorithm that we want to try 7 different values of $\lambda$ (which are generated for us).

In [ ]:
first_sweep=GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                        fairness_metric=DemographicParity(),
                        quality_metric=SimpleClassificationQualityMetric())

first_sweep.fit(X, Y, protected_attribute=A, number_of_lagrange_multipliers=11)

We can examine the values of $\lambda_i$ chosen for us:

In [ ]:
lagrange_multipliers = [x["lagrange_multiplier"] for x in (first_sweep.all_models)]
lagrange_multipliers

And we can look at how the weight the models place on the protected attribute (recall that in the fair case, this would be zero) varies with $\lambda_i$

In [ ]:
first_sweep_protected_attribute_weights = [
            x["model"].coef_[0][1] for x in first_sweep.all_models]
plt.scatter(lagrange_multipliers, first_sweep_protected_attribute_weights)
plt.xlabel("Lagrange Multiplier")
plt.ylabel("Weight of Protected Attribute in Model")
plt.show()

Since the fair case would have zero weight on the protected attribute and the above function is monotonic, we can readily select the best model by looking for those where that weight is closest to zero. For the sake of simplicity, we will just take the first model where that weight is greater than zero, and examine the weights in the model.

In [ ]:
first_sweep.best_model["lagrange_multiplier"]

We can also generate predictions from this model. When we plot them, we see that we're much closer to having a fair model, with both thresholds now somewhere close to 0.4.

In [ ]:
Y_first_predict = first_sweep.predict(X)
plot_data(X, Y_first_predict)

We can do better. For this simple case, we can zoom in on value of $\lambda$ based on the search we did above to find the first $\lambda$ where the weight on the protected attribute was greater than zero. Defining this as $\lambda_i$, we can do a finer grid search based on the range $[\lambda_{i-1}, \lambda_i]$. This time, we generate the $\lambda$ values we desire and pass them to the grid search:

In [ ]:
l = first_sweep.best_model["lagrange_multiplier"]
second_sweep_multipliers = np.linspace(l-0.5, l+0.5, 31)

second_sweep=GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                        fairness_metric=DemographicParity(),
                        quality_metric=SimpleClassificationQualityMetric())

second_sweep.fit(X, Y, protected_attribute=A, lagrange_multipliers=second_sweep_multipliers)

Once more we can plot the weight placed on the protected attribute as a function of $\lambda$. And afterwards, we can find the first model where that weight is greater than zero.

In [ ]:
second_sweep_protected_attribute_weights = [
            x["model"].coef_[0][1] for x in second_sweep.all_models]
second_sweep_lagrange_multipliers = [x["lagrange_multiplier"] for x in second_sweep.all_models]
plt.scatter(second_sweep_lagrange_multipliers, second_sweep_protected_attribute_weights)
plt.xlabel("Lagrange Multiplier")
plt.ylabel("Weight of Protected Attribute in Model")
plt.show()


We can extract the corresponding model and look at the weights

In [ ]:
second_sweep.best_model["lagrange_multiplier"]
second_sweep.best_model["model"].coef_

And finally, we can obtain a fresh set of predictions from this model. We can see that there is even less difference in the thresholds for the two different subpopulations.

In [ ]:
Y_second_predict = second_sweep.predict(X)
plot_data(X, Y_second_predict)